In [1]:
import numpy as np
from manim import *
config.media_width = "60%"

Manim Community v0.10.0

In [2]:
import cv2

In [8]:
nll_image = cv2.imread("nll1.png")[:, :, ::-1]

In [81]:
%%manim -v WARNING -qm Slides04Inference

class Slides04Inference(Scene):
    
    def create_y_labels(self, plane: NumberPlane):
        y_labels = ["A", "C", "B", "A"]
        
        self.y_label_group = []
        for i, yl in enumerate(y_labels):
            text = Tex(yl).scale(0.5)
            text.next_to(plane.coords_to_point(0, i + 0.5), LEFT * 0.5)
            self.y_label_group.append(text)
        self.y_label_group.reverse()
        
    def create_x_labels(self, plane: NumberPlane):
        x_labels = list(range(1, 28))
        
        self.x_label_group = []
        for i, xl in enumerate(x_labels):
            text = Tex(xl).scale(0.5)
            text.next_to(plane.coords_to_point(i + 0.5, 0), DOWN * 0.5)
            self.x_label_group.append(text)
            
    def create_probabilities_and_plane(self):
        self.nll = ImageMobject(nll_image, scale_to_resolution=700)
        self.nll.set_resampling_algorithm(RESAMPLING_ALGORITHMS["nearest"])
        
        self.plane = NumberPlane(
            x_range=[0, 27.01, 1],
            y_range=[0, 4.01, 1],
            x_length=self.nll.width,
            y_length=self.nll.height,
            axis_config={
                "stroke_color": BLUE,
                "stroke_width": 2,
            },
            background_line_style={
                "stroke_color": BLUE,
                "stroke_width": 2,
            }
        )        
        
        self.probabilities = Group(self.nll, self.plane)
    
    def construct(self):
        title = Title("Inference In Action Segmentation")
        self.create_probabilities_and_plane()

        self.add(title, self.probabilities)
        
        self.create_y_labels(self.plane)
        self.create_x_labels(self.plane)
        
        self.play(FadeIn(VGroup(*self.y_label_group)))
        self.play(FadeIn(VGroup(*self.x_label_group)))
        
        